# Importing libraries

In [1]:
!pip install tflearn 

     ---------------------------------------- 0.0/107.3 kB ? eta -:--:--
     ---------------------------------------- 0.0/107.3 kB ? eta -:--:--
     ---------------------------------------- 0.0/107.3 kB ? eta -:--:--
     -------------- ------------------------ 41.0/107.3 kB 2.0 MB/s eta 0:00:01
     ----------------- ------------------- 51.2/107.3 kB 871.5 kB/s eta 0:00:01
     ----------------- ------------------- 51.2/107.3 kB 871.5 kB/s eta 0:00:01
     ---------------------------- -------- 81.9/107.3 kB 512.0 kB/s eta 0:00:01
     ---------------------------------- - 102.4/107.3 kB 590.8 kB/s eta 0:00:01
     ------------------------------------ 107.3/107.3 kB 443.0 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127292 sha256=019a5c8f79fa47cfe040eaad73caa6745b9afd33d8c85ee8f707da7a918d3f30
  Stored in directory: c:\users\rutik\appdata\

In [2]:
#Used in Tensorflow Model 
import tensorflow as tf
import tflearn
import random
import numpy as np 

#Other
import json # for intent
import pickle # for saving model
import warnings
warnings.filterwarnings('ignore')


Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)



ImportError: cannot import name 'is_sequence' from 'tensorflow.python.util.nest' (C:\Users\rutik\anaconda\envs\Ruth\Lib\site-packages\tensorflow\python\util\nest.py)

## Contextualisation and other NLP Tasks.

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

## Loading intents

In [ ]:
print("Processing the Intents.....")
with open('intents.json') as json_data:
    intents = json.load(json_data)

## Generating words, classes, documents and ignore_words from intents

In [ ]:
import nltk

words = []
classes = []
documents = []
ignore_words = ['?']

print("Looping through the Intents to Convert them to words, classes, documents, and ignore_words....")
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # Tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # Add to our words list
        words.extend(w)
        # Add to documents in our corpus
        documents.append((w, intent['tag']))
        # Add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])


## Lemmatization, Lowering and remove duplicates

In [ ]:
print("Lemmatization, Lowering.......")
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
words

## Removing Duplicates

In [ ]:
# sort classes
classes = sorted(list(set (classes)))

# documents = combination between patterns and intents
print (len(documents), "documents")

# classes = intents
print (len(classes), "classes", classes)

# words = all words, vocabulary
print (len(words), "unique words", words)

## Creating training set and bag of words

In [ ]:
print("Creating the Data for our Model....")
training = []
output = []

In [ ]:
print("Creating an List (Empty) for Output.....")
output_empty = [0] * len(classes)

In [ ]:
print("Creating Traning Set, Bag of Words for our Model....")
for doc in documents:
    # initialize our bag of words
    bag = []
    #List of tokenize words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
        
    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])


In [ ]:
doc[0][1]

## Converting training set into numpy array

In [ ]:
print("Shuffling Randomly and Converting into Numpy Array for Faster Processing......")
random.shuffle(training)
training = np.array(training)
training

In [ ]:
print("Creating Train and Test Lists.....")
train_x = list(training[:,0])
train_y= list(training[:,1])
print("Building Neural Network for Out Chatbot to be Contextual.. ")

In [ ]:
print("Resetting graph data....")
tf.keras.backend.clear_session()

In [ ]:
# for multi-class classification tasks.
net = tflearn.input_data(shape=[None, len(train_x[0])])
# 1st number of training examples and 2nd number of input neurons
net = tflearn.fully_connected (net, 8)
# fully connected layers to the neural network
net = tflearn.fully_connected (net, 8)
net = tflearn.fully_connected (net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)
# output layer with a softmax activation function
print("Training....")

In [ ]:
model = tflearn.DNN (net, tensorboard_dir='tflearn_logs')

## Training model and saving the model

In [ ]:
print("Training the Model.......")
model.fit(train_x, train_y, n_epoch=500, batch_size=8, show_metric=True)

In [ ]:
print("Saving the Model.......")
model.save('model.h5')

In [ ]:
print("Pickle is also Saved........")
pickle.dump({'texts.pkl': words, 'labels.pkl':classes,
             'train_x':train_x, 'train_y':train_y},
            open("training_data","wb"))

## Loading pickle

In [ ]:
print("Loading Pickle.....")
data = pickle.load(open("training_data", "rb" ) )
words = data['texts.pkl']
classes = data['labels.pkl']
train_x = data['train_x']
train_y= data['train_y']

## Loading the Model

In [ ]:
# Load our saved model
#loaded_model = tf.keras.models.load_model('model.tflearn')
model.load('./model.h5')

## defining functions

In [ ]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array.
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

In [ ]:
# Return the Array of Bag of Words: True or False and 0 or 1 for each word of bag that exists in the Sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate (words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

In [ ]:
ERROR_THRESHOLD = 0.95
print("ERROR_THRESHOLD = 0.95")

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    results = [[i, r] for i, r in enumerate(res)]

    # Check if any prediction is above the threshold
    predictions_above_threshold = [r for r in results if r[1] > ERROR_THRESHOLD]

    # If there are predictions above threshold, sort them by strength of probability
    if predictions_above_threshold:
        predictions_above_threshold.sort(key=lambda x: x[1], reverse=True)
        return_list = [{"intent": classes[r[0]], "probability": str(r[1])} for r in predictions_above_threshold]
    else:
        # No predictions above threshold, return "Could not understand" response
        return_list = [{"intent": "invalid", "probability": "N/A"}]

    return return_list

# ERROR_THRESHOLD = 0.95
# print("ERROR_THRESHOLD = 0.95")

# def predict_class(sentence, model):
#     # filter out predictions below a threshold
#     p = bow(sentence, words,show_details=False)
#     res = model.predict(np.array([p]))[0]
#     results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
#     # sort by strength of probability
#     results.sort(key=lambda x: x[1], reverse=True)
#     return_list = []
#     for r in results:
#         return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
#     return return_list

In [ ]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

# def response(sentence,show_details=False):
#     results = predict_class(sentence)
#     # That Means if classification is Done then Find the Matching Tag.
#     if results:
#         # Long Loop to get the Result.
#         while results:
#             for i in intents['intents']:
#                 #Tag Finding
#                 if i['tag'] == results[0][0]:
#                     # Random Response from High Order Probabilities
#                     return print("Bot : ",random.choice(i['responses']))
                
#             results.pop(0)    

In [ ]:
def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

In [ ]:
from flask import Flask, render_template, request

app = Flask(__name__)
app.static_folder = 'static'

@app.route("/")
def home():
    return render_template("index.html")

@app.route("/get")
def get_bot_response():
    userText = request.args.get('msg')
    return chatbot_response(userText)


if __name__ == "__main__":
    app.run()